# ML BLOCKS

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import random_split

from torch_geometric.data import Data, DataLoader, Batch

from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm
from pathlib import Path
import gzip
import json

from warm_start import prepare_statistics, prepare_xy, prepare_gru_dataset, GruEDModel, make_gru_loader
from redundant_constraints import EdgeModel, GraphTrainer, build_grid_graph, plot_grid_graph, compute_dc_power_flow, prepare_powerflow_dataset

In [3]:
if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

print(f"Using device: {DEVICE}")

Using device: mps


# CHOOSE CASE

In [4]:
CASE = "case300"
OUT_DIR = Path("ml_results")

In [5]:
INITIAL_DATA_DIR = Path("dataset/initial_cases") / CASE
HIST_DATA_DIR = Path("dataset/solution_gurobi_cases") / CASE

print("Number of files:", len(list(HIST_DATA_DIR.glob('*.npz'))))

initial_files = sorted(INITIAL_DATA_DIR.glob("*.json.gz"))
hist_files = sorted(HIST_DATA_DIR.glob("*.npz"))

# Mistake in data
initial_files = [f for f in initial_files if "2017-09-04" not in f.name]

with gzip.open(initial_files[0], "rt", encoding="utf-8") as gzfile:
    first_initial_data = json.load(gzfile)
print("Initial data keys:", list(first_initial_data.keys()))
    
with np.load(hist_files[0], allow_pickle=True) as npzfile:
    first_hist_data = npzfile
print("Historical data keys:", first_hist_data.files)

# Create and plot a graph from the CASE data
G, bus_idx = build_grid_graph(first_initial_data)

# plot_grid_graph(G, bus_idx)

Number of files: 364
Initial data keys: ['SOURCE', 'Parameters', 'Generators', 'Transmission lines', 'Contingencies', 'Buses', 'Reserves']
Historical data keys: ['Thermal production (MW)/g1', 'Thermal production (MW)/g2', 'Thermal production (MW)/g3', 'Thermal production (MW)/g4', 'Thermal production (MW)/g5', 'Thermal production (MW)/g6', 'Thermal production (MW)/g7', 'Thermal production (MW)/g8', 'Thermal production (MW)/g9', 'Thermal production (MW)/g10', 'Thermal production (MW)/g11', 'Thermal production (MW)/g12', 'Thermal production (MW)/g13', 'Thermal production (MW)/g14', 'Thermal production (MW)/g15', 'Thermal production (MW)/g16', 'Thermal production (MW)/g17', 'Thermal production (MW)/g18', 'Thermal production (MW)/g19', 'Thermal production (MW)/g20', 'Thermal production (MW)/g21', 'Thermal production (MW)/g22', 'Thermal production (MW)/g23', 'Thermal production (MW)/g24', 'Thermal production (MW)/g25', 'Thermal production (MW)/g26', 'Thermal production (MW)/g27', 'Thermal p

In [6]:
# DATASET
N_total = len(list(HIST_DATA_DIR.glob('*.npz')))
N_train = int(0.7 * N_total)
N_val = int(0.15 * N_total)
N_test = N_total - N_train - N_val

# Split
train_init = initial_files[:N_train]
train_hist = hist_files[:N_train]

val_init = initial_files[N_train:N_train + N_val]
val_hist = hist_files[N_train:N_train + N_val]

test_init = initial_files[N_train + N_val:]
test_hist = hist_files[N_train + N_val:]

# WARM START

In [7]:
OUT_DIR_WARMSTART = OUT_DIR / "warm_start" / CASE
OUT_DIR_WARMSTART.mkdir(parents=True, exist_ok=True)

## Unit commitment

In [8]:
df_train = prepare_statistics(train_init, train_hist)
df_val = prepare_statistics(val_init, val_hist)
df_test = prepare_statistics(test_init, test_hist)

100%|██████████| 56/56 [00:02<00:00, 21.51it/s]


In [9]:
X_train, y_train = prepare_xy(df_train)
X_val, y_val = prepare_xy(df_val)
X_test, y_test = prepare_xy(df_test)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [10]:
uc_model = CatBoostClassifier(
    iterations=100,
    depth=6,
    learning_rate=0.1,
    loss_function='Logloss',
    verbose=20)

uc_model.fit(X_train_scaled, y_train, eval_set=(X_val_scaled, y_val))

y_pred_test = uc_model.predict(X_test_scaled)
print(classification_report(y_test, y_pred_test))

y_pred_val = uc_model.predict(X_val_scaled)

0:	learn: 0.6024290	test: 0.6093752	best: 0.6093752 (0)	total: 81.1ms	remaining: 8.03s
20:	learn: 0.0863293	test: 0.1270188	best: 0.1270188 (20)	total: 498ms	remaining: 1.87s
40:	learn: 0.0313637	test: 0.0789379	best: 0.0789379 (40)	total: 915ms	remaining: 1.32s
60:	learn: 0.0217320	test: 0.0770993	best: 0.0752293 (53)	total: 1.29s	remaining: 826ms
80:	learn: 0.0186351	test: 0.0762822	best: 0.0752293 (53)	total: 1.66s	remaining: 389ms
99:	learn: 0.0172498	test: 0.0770066	best: 0.0752293 (53)	total: 2.01s	remaining: 0us

bestTest = 0.07522929241
bestIteration = 53

Shrink model to first 54 iterations.
              precision    recall  f1-score   support

         0.0       0.70      0.89      0.78     10809
         1.0       0.99      0.97      0.98    128295

    accuracy                           0.96    139104
   macro avg       0.85      0.93      0.88    139104
weighted avg       0.97      0.96      0.96    139104



## Save warm start results

In [ ]:
# N_gen = len(df_val["gen_id"].unique())
# N_days = len(val_init)

# idx = 0

# for i in range(N_days):
#     day_name = Path(Path(val_init[i]).stem).stem
    
#     predictions = []
#     for g in range(N_gen):
#         for h in range(N_hours):
#             row = df_val.iloc[idx]
#             predictions.append({
#                 "gen_id": int(row["gen_id"]),
#                 "hour": int(row["hour"]),
#                 "is_on_pred": int(y_pred_val[idx])
#             })
#             idx += 1

#     output = {
#         "date": day_name,
#         "predictions": predictions
#     }

#     out_path = OUT_DIR / f"{day_name}.json"
#     with open(out_path, "w") as f:
#         json.dump(output, f, indent=2)

N_hours = 36       
N_gen = len(df_test["gen_id"].unique())
N_days = len(test_init)

idx = 0

for i in range(N_days):
    day_name = Path(Path(test_init[i]).stem).stem
    
    predictions = []
    for g in range(N_gen):
        for h in range(N_hours):
            row = df_test.iloc[idx]
            predictions.append({
                "gen_id": int(row["gen_id"]),
                "hour": int(row["hour"]),
                "is_on_pred": int(y_pred_test[idx])
            })
            idx += 1

    output = {
        "date": day_name,
        "predictions": predictions
    }

    out_path = OUT_DIR_WARMSTART / f"{day_name}.json"
    with open(out_path, "w") as f:
        json.dump(output, f, indent=2)

## Energy dispatch

In [12]:
X_seq_train, Y_train_seq = prepare_gru_dataset(train_init, train_hist, y_train.values)
X_seq_val, Y_val_seq = prepare_gru_dataset(val_init, val_hist, y_val.values)
X_seq_test, Y_test_seq = prepare_gru_dataset(test_init, test_hist, y_pred_test)

100%|██████████| 56/56 [00:01<00:00, 43.85it/s]


In [13]:
X_static_train = df_train[df_train["hour"] == 0][["gen_id", "pmin", "pmax", "avg_cost_per_mw",
                                                  "avg_startup_cost_per_hour", "startup_delay_min", "startup_delay_max", 
                                                  "ramp_up", "ramp_down", "startup_limit", "shutdown_limit",
                                                  "min_uptime", "min_downtime"
                                                  ]].values

X_static_val = df_val[df_val["hour"] == 0][["gen_id", "pmin", "pmax", "avg_cost_per_mw",
                                            "avg_startup_cost_per_hour", "startup_delay_min", "startup_delay_max", 
                                            "ramp_up", "ramp_down", "startup_limit", "shutdown_limit",
                                            "min_uptime", "min_downtime"
                                            ]].values
X_static_test = df_test[df_test["hour"] == 0][["gen_id", "pmin", "pmax", "avg_cost_per_mw",
                                                "avg_startup_cost_per_hour", "startup_delay_min", "startup_delay_max", 
                                                "ramp_up", "ramp_down", "startup_limit", "shutdown_limit",
                                                "min_uptime", "min_downtime"
                                                ]].values

In [14]:
scaler_static = StandardScaler()

X_static_train_scaled = scaler_static.fit_transform(X_static_train)
X_static_val_scaled = scaler_static.transform(X_static_val)
X_static_test_scaled = scaler_static.transform(X_static_test)

In [15]:
train_loader = make_gru_loader(X_seq_train, X_static_train_scaled, Y_train_seq)
val_loader = make_gru_loader(X_seq_val, X_static_val_scaled, Y_val_seq, shuffle=False)
test_loader = make_gru_loader(X_seq_test, X_static_test_scaled, Y_test_seq, shuffle=False)

In [16]:
model = GruEDModel(seq_input_dim=3, static_input_dim=X_static_train.shape[1]).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

In [ ]:
# EPOCHS = 20

# for epoch in range(EPOCHS):
#     model.train()
#     train_loss = 0.0
#     for x_seq, x_static, y in train_loader:
#         x_seq, x_static, y = x_seq.to(DEVICE), x_static.to(DEVICE), y.to(DEVICE)
#         optimizer.zero_grad()
#         output = model(x_seq, x_static)
#         loss = criterion(output, y)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item() * x_seq.size(0)

#     model.eval()
#     val_loss = 0.0
#     with torch.no_grad():
#         for x_seq, x_static, y in val_loader:
#             x_seq, x_static, y = x_seq.to(DEVICE), x_static.to(DEVICE), y.to(DEVICE)
#             output = model(x_seq, x_static)
#             loss = criterion(output, y)
#             val_loss += loss.item() * x_seq.size(0)

#     print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss/len(train_loader.dataset):.4f} | Val Loss: {val_loss/len(val_loader.dataset):.4f}")

tensor([[[ 0.0000e+00,  1.0000e+00,  1.0000e+00,  ..., -1.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00,  1.0000e+00,  ..., -1.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00,  1.0000e+00,  ..., -1.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [        nan,         nan,         nan,  ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan,  ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan,  ...,         nan,
                  nan,         nan]],

        [[ 0.0000e+00,  1.0000e+00,  1.0000e+00,  ..., -1.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00,  1.0000e+00,  ..., -1.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00,  1.0000e+00,  ..., -1.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [        nan,   

KeyboardInterrupt: 

In [ ]:
# model.eval()
# test_preds = []
# test_targets = []

# with torch.no_grad():
#     for x_seq, x_static, y in test_loader:
#         x_seq, x_static = x_seq.to(device), x_static.to(device)
#         output = model(x_seq, x_static)
#         test_preds.append(output.cpu())
#         test_targets.append(y)

# preds = torch.cat(test_preds).numpy()
# targets = torch.cat(test_targets).numpy()

# mse = np.mean((preds - targets) ** 2)
# print("Test MSE:", mse)


# IDENTIFY REDUNDANT CONSTRAINTS

In [23]:
OUT_DIR_REDUNDANTCONS = OUT_DIR / "redundant_constraints" / CASE
OUT_DIR_REDUNDANTCONS.mkdir(parents=True, exist_ok=True)

In [18]:
# Power flow on each transmission line does not exceed its thermal limits
LIMIT = 5.0

train_set = prepare_powerflow_dataset(train_init, train_hist, case=CASE, limit=LIMIT)
val_set = prepare_powerflow_dataset(val_init, val_hist, case=CASE, limit=LIMIT)
test_set = prepare_powerflow_dataset(test_init, test_hist, case=CASE, limit=LIMIT)

loader_train = DataLoader(train_set, batch_size=16, shuffle=True)
loader_val = DataLoader(val_set, batch_size=16)
loader_test = DataLoader(test_set, batch_size=len(test_set))


100%|██████████| 56/56 [00:00<00:00, 92.49it/s] 
/Users/grigorii/Library/Python/3.9/lib/python/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:


# # Statistic data for topology
# records = []

# for init_f, hist_f in tqdm(zip(initial_files, hist_files), total=len(hist_files)):
#     with gzip.open(init_f, "rt", encoding="utf-8") as f:
#         data = json.load(f)
        
#     buses = data["Buses"]
#     lines = data["Transmission lines"]
#     gens  = data["Generators"]
    
#     n_bus = len(buses)
#     n_line = len(lines)
    
#     bus_ids = list(buses.keys())
#     line_ids = list(lines.keys())
#     bus_idx = {bus_id: i for i, bus_id in enumerate(bus_ids)}
    
#     # Transmission lines FEATURES (topology) - edge
#     edge_index_list = []
#     edge_attr_list = []

#     for line_id in line_ids:
#         line = lines[line_id]
#         i = bus_idx[line["Source bus"]]
#         j = bus_idx[line["Target bus"]]
#         reactance = line["Reactance (ohms)"]
#         susceptance = line["Susceptance (S)"]

#         edge_index_list.append([i, j])
#         edge_attr_list.append([reactance, susceptance])

#     edge_index = torch.tensor(edge_index_list).T
#     edge_attr = torch.tensor(edge_attr_list, dtype=torch.float32)
    
#     # Buses features
#     x = []

#     for bus_id in bus_ids:
#         bus = buses[bus_id]
#         load = bus.get("Load (MW)")

#         mean_load = np.mean(load)
#         max_load = np.max(load)
#         min_load = np.min(load)
#         std_load = np.std(load)

#         x.append([mean_load, max_load, min_load, std_load])

#     x = torch.tensor(x, dtype=torch.float32)

#     # TODO: Calculate power flow, take from historical data
#     # with np.load(hist_f, allow_pickle=True) as npzfile:
#         # hist_data = npzfile
    
#     flows = compute_dc_power_flow(data)
    
#     y = (flows >= LIMIT).astype(np.int64)
#     y = torch.tensor(y, dtype=torch.float32)
#     day_id = init_f.stem 
    
#     data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
#     data.case_id = CASE
#     data.day_id  = day_id
#     data.edge_id = line_ids 
#     records.append(data)

100%|██████████| 364/364 [00:03<00:00, 105.20it/s]


# ML model

In [20]:
model = EdgeModel(
    node_in_dim=train_set[0].x.size(1),
    edge_in_dim=train_set[0].edge_attr.size(1)
    ).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
bce_loss = nn.BCELoss()
EPOCHS = 100

trainer = GraphTrainer(
    model=model,
    optimizer=optimizer,
    criterion=bce_loss,
    loader_train=loader_train,
    loader_val=loader_val,
    epochs = EPOCHS,
    device = DEVICE
    )

trainer.fit()

Epoch 000 | loss=1.8979 | acc=0.607 | prec=0.881 | rec=0.576 | f1=0.697
Epoch 001 | loss=1.2414 | acc=0.704 | prec=0.852 | rec=0.753 | f1=0.800
Epoch 002 | loss=1.0350 | acc=0.730 | prec=0.850 | rec=0.797 | f1=0.823
Epoch 003 | loss=0.8081 | acc=0.746 | prec=0.863 | rec=0.805 | f1=0.833
Epoch 004 | loss=0.5361 | acc=0.697 | prec=0.884 | rec=0.706 | f1=0.785
Epoch 005 | loss=0.4598 | acc=0.662 | prec=0.884 | rec=0.655 | f1=0.753
Epoch 006 | loss=0.4408 | acc=0.712 | prec=0.884 | rec=0.729 | f1=0.799
Epoch 007 | loss=0.4307 | acc=0.714 | prec=0.892 | rec=0.724 | f1=0.799
Epoch 008 | loss=0.4239 | acc=0.727 | prec=0.896 | rec=0.738 | f1=0.809
Epoch 009 | loss=0.4201 | acc=0.733 | prec=0.899 | rec=0.744 | f1=0.814
Epoch 010 | loss=0.4164 | acc=0.741 | prec=0.905 | rec=0.749 | f1=0.819
Epoch 011 | loss=0.4131 | acc=0.745 | prec=0.907 | rec=0.752 | f1=0.822
Epoch 012 | loss=0.4106 | acc=0.748 | prec=0.907 | rec=0.756 | f1=0.825
Epoch 013 | loss=0.4076 | acc=0.753 | prec=0.907 | rec=0.763 | f

In [ ]:
model.eval()
threshold = 0.7

with torch.no_grad():
    for batch in tqdm(loader_test):
        for graph in batch.to_data_list():
            graph = graph.to(DEVICE)

            y_prob = model(graph.x, graph.edge_index, graph.edge_attr).cpu()
            y_pred = (y_prob > threshold).int()

            # Инвертируем 0 ↔ 1
            line_dict = {lid: int(1 - bit) for lid, bit in zip(graph.edge_id, y_pred.tolist())}

            payload = {
                "day": graph.day_id,
                "line_pruning": line_dict
            }

            out_path = OUT_DIR_REDUNDANTCONS / f"{graph.day_id}.json"
            with open(out_path, "w") as f:
                json.dump(payload, f, indent=2)

100%|██████████| 1/1 [00:00<00:00,  4.54it/s]
